# This Notebook supports the 2023 August 03 talk given at the Silicon Valley AAII Investment Club meeting.  
This NB is forked from brk_val_2.0.ipynb.  

In [43]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import datetime as dt
import math

# Current date
# This is used as "base" date for CPI adjustments
cur_date = '2023-07-28'

# Max rows and cols to display
pd.set_option("display.max_rows",130)
pd.set_option("display.max_columns",130)

### Build DF to hold data
First we'll build a DF that has all dates, including non-market days, going back to 2000-03-31 (date of 1st book value).   
We'll then merge our data into this DF.

In [44]:
# Start and end dates
# Note that the BRK data starts on 1999-03-31, but the CPI monthly data series has entries on the 15th of each month.
# In order to get CPI data for 1999-03-31 thru 1999-04-14, which we need, we'll need to make a date for the 1999-03-15 
# CPI data and then forward fill. In general, the start date should be at or prior to the 15th prior to the earliest
# EOQ data.
startDate = dt.datetime(1999, 3, 15)
endDate = dt.datetime(2023, 12, 31)  # This date can extend beyond the current date

# pd.date_range is exactly what we need
data = pd.DataFrame()
data['date'] = pd.date_range(startDate, endDate)

### Process CPI data, merge to data DF
https://www.bls.gov/regions/mid-atlantic/data/consumerpriceindexhistorical_us_table.htm  
Prices used to compute the CPI are collected during the entire month. CPI data is published monthly, with the 
index value representing an estimate of the price level for the month as a whole, rather than a specific date. 
This suggests using the middle of the month for CPI values. 
https://www.bls.gov/cpi/questions-and-answers.htm#:~:text=Prices%20used%20to%20compute%20the,rather%20than%20a%20specific%20date.   


In [45]:
# Build dates to hold CPI data
# We'll use the middle of the month, as the CPI is calculated on values collected during the entire month.
import calendar

dates = []
for yr in range(1999,2024):
    for mo in range(1,13):
        # Get last day of month, based on year an month
        # monthrange() returns day of week for first day of month (discard) and days in month, taking into account leap years.
        # _, day = calendar.monthrange(yr, mo)        
        date = dt.datetime(yr,mo,15)
        #print(date)
        dates.append(date)

cpi = pd.DataFrame()
cpi['date'] = dates
cpi.tail(3)

,date
297,2023-10-15
298,2023-11-15
299,2023-12-15


#### Read and Process CPI data

In [46]:
# Read in cpi data in form of a table (yr = row, month = col)
cpi_tbl = pd.read_csv('cpi_jun_2023.csv')

# Drop year, flatten, and then concat data with cpi.
# Note concat does not need lengths to be equal
cpi_tbl.drop('Year', inplace=True, axis=1)
cpi =  pd.concat([cpi, pd.DataFrame(cpi_tbl.to_numpy().flatten(), columns=['cpi'])], axis=1)

# Determine monthly cpi % change as a proxy to forecast future cpi's at bottom of DF (e.g. thru current year)
# Divide the last non-NaN cpi value by the first cpi value and then take the root using number of months
cpi_chg_mo = (cpi.loc[ ~cpi.cpi.isnull() ].iloc[-1].cpi / cpi.iloc[0].cpi)**(1/cpi.loc[ ~cpi.cpi.isnull() ].shape[0])

# Now forecast future cpi's
# Each future month's cpi is forecast as the previous month's cpi * cpi_chg_mo
for idx in cpi.loc[ cpi.cpi.isnull() ].index:
    cpi.at[idx, 'cpi'] = cpi_chg_mo * cpi.at[idx-1, 'cpi']
cpi.tail(3)

,date,cpi
297,2023-10-15,307.689304
298,2023-11-15,308.337782
299,2023-12-15,308.987627


In [29]:
cpi.head(2)

,date,cpi
0,1999-01-15,164.3
1,1999-02-15,164.5


#### Merge CPI data to data DF
Use linear interpolation to fill in missing CPI values   
Note CPI data goes back prior to 1999-03-15, but the left merge will just leave off the earlier, unneeded CPI values.

In [47]:
data = data.merge(cpi, how='left', left_on='date', right_on='date')

# Use linear interpolation to fill in missing CPI values
data['cpi'] = data['cpi'].interpolate(method='linear', limit_direction='forward', axis=0)
data.head()

,date,cpi
0,1999-03-15,165.000000
1,1999-03-16,165.038710
2,1999-03-17,165.077419
3,1999-03-18,165.116129
4,1999-03-19,165.154839


### Process EOQ dates, Release dates, BV - Merge into data DF
I've entered the release date as always occuring on a Saturday, although this is not always true.   
Earnings have also commonly been released Friday after market close, and possible on a different weekday once or twice (not sure about that).

In [48]:
# Read in end-of-quarter dates, earnings release dates and BV data.
eoq_data = pd.read_csv('brk_bv_flat.csv', parse_dates=['eoq_dt', 'release_dt'])
eoq_data.sort_values('eoq_dt', ascending=True, ignore_index=True, inplace=True)

# Each B share is worth 1/1500 of an A share
eoq_data['bv_b'] = eoq_data.bv_a / 1500

eoq_data.tail(2)

,eoq_dt,release_dt,bv_a,bv_b
95,2022-12-31,2023-02-25,323593,215.728667
96,2023-03-31,2023-05-06,347932,231.954667


In [34]:
eoq_data.head(2)

,eoq_dt,release_dt,bv_a,bv_b
0,1999-03-31,1999-05-08,38097,25.398000
1,1999-06-30,1999-08-07,38189,25.459333


#### Merge BV data into data DF

In [49]:
data = data.merge(eoq_data, how='left', left_on='date', right_on='eoq_dt')
data.tail(3)

,date,cpi,eoq_dt,release_dt,bv_a,bv_b
9055,2023-12-29,308.987627,NaT,NaT,NaN,NaN
9056,2023-12-30,308.987627,NaT,NaT,NaN,NaN
9057,2023-12-31,308.987627,NaT,NaT,NaN,NaN


#### Adjust BV for CPI
Current date entered in first cell

In [50]:
# Finds 1st occurance of row w/ given date (there should only be 1!) and returns corresponding CPI
data['bv_b_cpi'] = data['cpi'].loc[ data.date == cur_date].iloc[0] * data.bv_b / data.cpi 

#### Apply natural log to CPI adjusted BV

In [51]:
data['bv_b_cpi_ln'] = data['bv_b_cpi'].apply(np.log)

### Apply Linear Regression to BV CPI Log series

In [52]:
# Linear regression model
lin_reg = LinearRegression()

# Construct pd.series to feed into regression model
x = data.loc[~data.bv_b.isnull()].index.values
y = data.loc[~data.bv_b.isnull(), 'bv_b_cpi_ln']

# Get length of time series
series_len = x.shape[0]

# Initialize array to hold slopes and intercepts from regression line
regression = np.empty((series_len, 2))
regression[:] = np.nan

# Number of quarters to use in the regression calculation
# 40 quarters = 10 years
window = 40
for row in range(window, series_len):
    x_window = x[row - window: row].reshape(-1,1)
    y_window = y[row - window: row]
    lin_reg.fit(x_window, y_window)      
    regression[row] = [lin_reg.coef_[0], lin_reg.intercept_]

# Add regression slope and intercept to y
# Convert y from series to DF so we can add columns
y = y.to_frame()
y['slope_b'] = regression[:,0]
y['intercept_b'] = regression[:,1]

#### Merge regression parameters onto data
Each set of regression parameters will be utilized between the release dates.  
So we want to merge the regression parameters into the release dates.   

In [53]:
# Add release dates to y
y = y.join(data['release_dt'], how='left')

# Merge regression parameters onto data on 'release_dt'
# This creates 2 release date cols: release_dt_x (orig) and release_dt_y (merged)
data = data.merge(y[['release_dt','slope_b','intercept_b']], how='left', left_on='date', right_on='release_dt')

# Forward Fill slope and intercept.
# This will fill in missing slopes/intercepts between release dates, which is exactly what we want.
data['slope_b'] = data['slope_b'].ffill()
data['intercept_b'] = data['intercept_b'].ffill()
data.head(2)

,date,cpi,eoq_dt,release_dt_x,bv_a,bv_b,bv_b_cpi,bv_b_cpi_ln,release_dt_y,slope_b,intercept_b
0,1999-03-15,165.00000,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,1999-03-16,165.03871,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN


#### Create on-trend BV for BRK-B

In [54]:
# Trend BV curve for cpi and ln adjusted brk-b
data['bv_b_ln_trend'] = data.index.values * data.slope_b + data.intercept_b

# Trend BV curve for cpi adjusted brk-b
data['bv_b_cpi_trend'] = data.bv_b_ln_trend.apply( np.exp )

In [23]:
# Test code
data.loc[~data.bv_a.isnull()].shape
data.tail(2)

,date,cpi,eoq_dt,release_dt_x,bv_a,bv_b,bv_b_cpi,bv_b_cpi_ln,release_dt_y,slope_b,intercept_b,bv_b_ln_trend,bv_b_cpi_trend
9056,2023-12-30,308.987627,NaT,NaT,NaN,NaN,NaN,NaN,NaT,0.000236,3.466555,5.602967,271.230017
9057,2023-12-31,308.987627,NaT,NaT,NaN,NaN,NaN,NaN,NaT,0.000236,3.466555,5.603203,271.294011


In [24]:
data.head(2)

,date,cpi,eoq_dt,release_dt_x,bv_a,bv_b,bv_b_cpi,bv_b_cpi_ln,release_dt_y,slope_b,intercept_b,bv_b_ln_trend,bv_b_cpi_trend
0,1999-03-15,165.00000,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,1999-03-16,165.03871,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


#### Download BRK prices, merge into dataframe

In [55]:
# Preference is to use pandas_datareader, but it is currently fubared.
import yfinance as yf

# Starting and end dates for collecting price data
# Uncomment if different than above
# Note, when using yfinance, enter the day following the date you want to retrieve data through.
# E.g. to retrieve data thru 2023 Jan 11 enter 2023-01-12 as the end date.
# Seems to be a quirk of yfinance.
startDate = dt.datetime(1999, 3, 31)
endDate = dt.datetime(2023, 8, 2)

brk_b = yf.download('brk-b', startDate, endDate)
brk_b.tail(2)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-07-31,350.730011,352.329987,350.209991,351.959991,351.959991,2621600
2023-08-01,352.029999,353.420013,351.250000,352.260010,352.260010,2293300


In [56]:
# Merge prices into dataframe
data = data.merge(brk_b[['Adj Close']], how='left', left_on='date', right_index=True)
data = data.rename(columns={'Adj Close': 'brk-b'})

In [57]:
# Adjust prices for CPI
data['b_cpi_adj_cl'] = data['cpi'].loc[ data.date == cur_date ].iloc[0] * data['brk-b'] / data.cpi

# Apply $ln()$ to cpi adjusted prices (log space)
data['b_cpi_adj_cl_ln'] = data['b_cpi_adj_cl'].apply(np.log)

# Forward fill adjusted close prices over non-mkt days for 1 year forward return calculations.
data['b_cpi_adj_cl_ffill'] = data['b_cpi_adj_cl'].ffill()

#### Download SPY prices, merge into dataframe

In [58]:
# Starting and end dates for collecting price data
# Uncomment if different than above
# Note, when using yfinance, enter the day following the date you want to retrieve data through.
# E.g. to retrieve data thru 2023 Jan 11 enter 2023-01-12 as the end date.
# Seems to be a quirk of yfinance.
startDate = dt.datetime(1999, 3, 31)
endDate = dt.datetime(2023, 8, 2)

spy = yf.download('spy', startDate, endDate)
spy.tail(2)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-07-31,457.410004,458.160004,456.049988,457.790009,457.790009,62040400
2023-08-01,456.269989,457.250000,455.489990,456.480011,456.480011,55291500


In [59]:
# Merge prices into dataframe
data = data.merge(spy[['Adj Close']], how='left', left_on='date', right_index=True)
data = data.rename(columns={'Adj Close': 'spy'})

In [60]:
# Adjust prices for CPI
data['spy_cpi_adj_cl'] = data['cpi'].loc[ data.date == cur_date ].iloc[0] * data['spy'] / data.cpi

# Apply $ln()$ to cpi adjusted prices (log space)
data['spy_cpi_adj_cl_ln'] = data['spy_cpi_adj_cl'].apply(np.log)

# Forward fill adjusted close prices over non-mkt days for 1 year forward return calculations.
data['spy_cpi_adj_cl_ffill'] = data['spy_cpi_adj_cl'].ffill()

### Calculate P(cpi adjusted) / on trend BV (cpi adjusted)

In [61]:
data['price_to_trend_bv'] = data['b_cpi_adj_cl'] / data['bv_b_cpi_trend']

### Analysis
Calculate mean and median price/BV ratio

In [62]:
mean_p_to_trend_bv = data['price_to_trend_bv'].mean()
median_p_to_trend_bv = data['price_to_trend_bv'].median()

# Calcuate time-series of mean and median P/trend BV
#data['mean_p_to_trend_bv_series'] = mean_p_to_trend_bv * data.bv_b_cpi_trend
#data['median_p_to_trend_bv_series'] = median_p_to_trend_bv * data.bv_b_cpi_trend

In [63]:
print(median_p_to_trend_bv)
mean_p_to_trend_bv

1.34785699026822


1.341256881674828

In [82]:
# Calcuate time-series of mean and median P/trend BV
# Mean
data['mean_p_to_trend_bv_series'] = data['price_to_trend_bv'].rolling(window=10**6, min_periods=1).mean()
data['mean_p_to_trend_bv_series'] = data.mean_p_to_trend_bv_series * data.bv_b_cpi_trend

# Median
data['median_p_to_trend_bv_series'] = data['price_to_trend_bv'].rolling(window=10**6, min_periods=1).median()
data['median_p_to_trend_bv_series'] = data.median_p_to_trend_bv_series * data.bv_b_cpi_trend

#### Calculate returns by quantile

In [65]:
yr = 365 # days

# BRK
data['rtn_1yr'] = data.b_cpi_adj_cl_ffill.shift(-yr) / data.b_cpi_adj_cl
data['rtn_2yr'] = data.b_cpi_adj_cl_ffill.shift(-2*yr) / data.b_cpi_adj_cl

# SPY
data['spy_rtn_1yr'] = data.spy_cpi_adj_cl_ffill.shift(-yr) / data.spy_cpi_adj_cl
data['spy_rtn_2yr'] = data.spy_cpi_adj_cl_ffill.shift(-2*yr) / data.spy_cpi_adj_cl

In [67]:
# Calculate quniteles
data['q_20'] = pd.qcut(data['price_to_trend_bv'], 20, labels=False)

quant_rtns = data[['q_20','rtn_1yr','spy_rtn_1yr','rtn_2yr','spy_rtn_2yr']].groupby( ['q_20'], axis=0 ).median()
quant_rtns['min_p2bv_cpi'] = data[['q_20', 'price_to_trend_bv']].groupby( ['q_20'], axis=0 ).min()
quant_rtns['max_p2bv_cpi'] = data[['q_20', 'price_to_trend_bv']].groupby( ['q_20'], axis=0 ).max()
quant_rtns['bdry_price_to_trend_bv'] = (quant_rtns.max_p2bv_cpi + quant_rtns.min_p2bv_cpi.shift(-1)) / 2
quant_rtns

,rtn_1yr,spy_rtn_1yr,rtn_2yr,spy_rtn_2yr,min_p2bv_cpi,max_p2bv_cpi,bdry_price_to_trend_bv
q_20,,,,,,,
0.0,1.325891,1.255584,1.504053,1.402491,0.963377,1.100149,1.100159
1.0,1.225961,1.161639,1.492741,1.409437,1.100170,1.145464,1.145504
2.0,1.220083,1.138696,1.461666,1.347428,1.145544,1.179056,1.179262
3.0,1.244347,1.132821,1.162007,1.188967,1.179468,1.204932,1.204975
4.0,1.223089,1.183871,1.220386,1.211512,1.205018,1.234607,1.234612
5.0,1.166887,1.174254,1.322516,1.270066,1.234617,1.264710,1.264777
6.0,1.070708,1.143749,1.249732,1.322285,1.264844,1.298122,1.298172
7.0,1.018647,1.088080,1.252401,1.305692,1.298221,1.316855,1.316856
8.0,1.047376,1.094598,1.184754,1.301317,1.316856,1.332917,1.332956


In [68]:
# Current price to book
cur_date = '2023-08-02'  # cur_date should be last market day, for which there is data. 
data['b_cpi_adj_cl'].loc[ data.date == cur_date ].iloc[0] / data['bv_b_cpi_trend'].loc[ data.date == cur_date ].iloc[0]

1.3302534575421627

### Plot BV trend curve (CPI adjusted)

In [101]:
import matplotlib.pyplot as plt
import numpy as np

# For interactive Matplotlib pop-out window
%matplotlib qt

x1 = data.loc[~data.bv_b_cpi_trend.isnull(), 'date']
y1 = data.loc[~data.bv_b_cpi_trend.isnull(), 'bv_b_cpi_trend']

x2 = data.loc[~data.bv_b_cpi_trend.isnull(), 'date']
y2 = data.loc[~data.bv_b_cpi_trend.isnull(), 'b_cpi_adj_cl']

x3 = data.loc[~data.mean_p_to_trend_bv_series.isnull(), 'date']
y3 = data.loc[~data.mean_p_to_trend_bv_series.isnull(), 'mean_p_to_trend_bv_series']

x4 = data.loc[~data.median_p_to_trend_bv_series.isnull(), 'date']
y4 = data.loc[~data.median_p_to_trend_bv_series.isnull(), 'median_p_to_trend_bv_series']

x5 = data.loc[~data.bv_b_cpi_trend.isnull() & ~data.bv_b_cpi.isnull(), 'date']
y5 = data.loc[~data.bv_b_cpi_trend.isnull() & ~data.bv_b_cpi.isnull(), 'bv_b_cpi']

plt.plot(x1, y1, color='black', linewidth=1)
plt.scatter(x2, y2, color='olive', s=1)
plt.plot(x3, y3, color='darkturquoise', linewidth=1)
plt.plot(x4, y4, color='aqua', linewidth=1)
plt.scatter(x5, y5, color='red', s=2)
plt.show()

### BRK vs SPY

In [105]:
dt = '2014-08-01'
b1  = data.loc[ data.date > dt ].loc[ ~data['brk-b'].isna(), 'date' ]
by1 = data.loc[ data.date > dt ].loc[ ~data['brk-b'].isna(), 'brk-b'] / data.loc[ data.date > dt 
                        ].loc[ ~data['brk-b'].isna(), 'brk-b'].iat[0]
s1  = data.loc[ data.date > dt ].loc[ ~data['spy'].isna(), 'date' ]
sy1 = data.loc[ data.date > dt ].loc[ ~data['spy'].isna(), 'spy'] / data.loc[ data.date > dt
                        ].loc[ ~data['spy'].isna(), 'spy'].iat[0]

plt.plot(b1, by1, color='blue', linewidth=1, label='BRK')
plt.plot(s1, sy1, color='green', linewidth=1, label='SPY')
plt.legend()
plt.show()

,date,cpi,eoq_dt,release_dt_x,bv_a,bv_b,bv_b_cpi,bv_b_cpi_ln,release_dt_y,slope_b,intercept_b,bv_b_ln_trend,bv_b_cpi_trend,Adj Close,b_cpi_adj_cl,b_cpi_adj_cl_ln,b_cpi_adj_cl_ffill,price_to_trend_bv
9058,2023-12-31,306.072923,NaT,NaT,NaN,NaN,NaN,NaN,NaT,0.00025,3.377037,5.640533,281.612869,NaN,NaN,NaN,310.183501,NaN


In [123]:
#data[[ 'bv_b_cpi','bv_b_cpi_ln' ]].loc[ ~data.bv_b_cpi.isnull() ]
#data.bv_b_cpi_ln.apply( np.exp ).loc[ ~data.bv_b_cpi.isnull() ]
data.loc[ (data.date > '2023-02-13') & (data.date < '2023-02-28') ]
#data.tail(2)

,date,cpi,eoq_dt,release_dt_x,bv_a,bv_b,bv_b_cpi,bv_b_cpi_ln,release_dt_y,slope_b,intercept_b,bv_b_ln_trend,bv_b_cpi_trend,Adj Close,b_cpi_adj_cl,b_cpi_adj_cl_ln,b_cpi_adj_cl_ffill,price_to_trend_bv,mean_p_to_trend_bv_series,median_p_to_trend_bv_series,rtn_1yr,rtn_1yr_avg,rtn_1yr_med,rtn_2yr,rtn_2yr_avg,rtn_2yr_med,q_20
8738,2023-02-14,299.771018,NaT,NaT,NaN,NaN,NaN,NaN,NaT,0.000250,3.356966,5.540498,254.804902,310.790009,311.018170,5.739851,311.018170,1.220613,342.300108,344.741314,NaN,NaN,NaN,NaN,NaN,NaN,4.0
8739,2023-02-15,299.791052,NaT,NaT,NaN,NaN,NaN,NaN,NaT,0.000250,3.356966,5.540748,254.868583,309.630005,309.836608,5.736045,309.836608,1.215672,342.385656,344.827472,NaN,NaN,NaN,NaN,NaN,NaN,4.0
8740,2023-02-16,299.813278,NaT,NaT,NaN,NaN,NaN,NaN,NaT,0.000250,3.356966,5.540998,254.932280,308.179993,308.362767,5.731277,308.362767,1.209587,342.471225,344.913651,NaN,NaN,NaN,NaN,NaN,NaN,4.0
8741,2023-02-17,299.835505,NaT,NaT,NaN,NaN,NaN,NaN,NaT,0.000250,3.356966,5.541248,254.995993,308.239990,308.399937,5.731397,308.399937,1.209431,342.556816,344.999852,NaN,NaN,NaN,NaN,NaN,NaN,4.0
8742,2023-02-18,299.857731,NaT,NaT,NaN,NaN,NaN,NaN,NaT,0.000250,3.356966,5.541498,255.059722,NaN,NaN,NaN,308.399937,NaN,342.642428,345.086075,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8743,2023-02-19,299.879958,NaT,NaT,NaN,NaN,NaN,NaN,NaT,0.000250,3.356966,5.541748,255.123466,NaN,NaN,NaN,308.399937,NaN,342.728061,345.172319,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8744,2023-02-20,299.902184,NaT,NaT,NaN,NaN,NaN,NaN,NaT,0.000250,3.356966,5.541997,255.187227,NaN,NaN,NaN,308.399937,NaN,342.813716,345.258584,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8745,2023-02-21,299.924411,NaT,NaT,NaN,NaN,NaN,NaN,NaT,0.000250,3.356966,5.542247,255.251004,302.720001,302.787302,5.713031,302.787302,1.186234,342.899392,345.344872,NaN,NaN,NaN,NaN,NaN,NaN,3.0
8746,2023-02-22,299.946637,NaT,NaT,NaN,NaN,NaN,NaN,NaT,0.000250,3.356966,5.542497,255.314796,303.160004,303.204933,5.714409,303.204933,1.187573,342.985090,345.431180,NaN,NaN,NaN,NaN,NaN,NaN,3.0
8747,2023-02-23,299.968864,NaT,NaT,NaN,NaN,NaN,NaN,NaT,0.000250,3.356966,5.542747,255.378604,303.070007,303.092464,5.714038,303.092464,1.186836,343.070809,345.517511,NaN,NaN,NaN,NaN,NaN,NaN,3.0


### LEGACY - Analysis in Log space
Calculate mean and median price/BV ratio in log space

In [21]:
data['p2bv_cpi_ln'] = data.b_cpi_adj_cl_ln / data.bv_b_ln_trend
mean_p2bv = data['p2bv_cpi_ln'].mean()
median_p2bv = data['p2bv_cpi_ln'].median()

In [22]:
data['mean_p2bv_ln'] = mean_p2bv * data.bv_b_ln_trend
data['median_p2bv_ln'] = median_p2bv * data.bv_b_ln_trend

We'll use a window over which we'll calculate an average forward return.

#### Calculate quantiles

In [315]:
data['q_20_ln'] = pd.qcut(data['p2bv_cpi_ln'], 20, labels=False)

In [316]:
quant_rtns = data[['q_20_ln', 'rtn_1yr_avg','rtn_2yr_avg']].groupby( ['q_20'], axis=0 ).median()
quant_rtns['max_p2bv_cpi_ln'] = data[['q_20_ln', 'p2bv_cpi_ln']].groupby( ['q_20'], axis=0 ).max()
quant_rtns['min_p2bv_cpi_ln'] = data[['q_20_ln', 'p2bv_cpi_ln']].groupby( ['q_20'], axis=0 ).min()
quant_rtns['bdry_p2bv_cpi_ln'] = (quant_rtns.max_p2bv_cpi_ln + quant_rtns.min_p2bv_cpi_ln.shift(-1)) / 2
quant_rtns['bdry_p2bv_cpi'] = quant_rtns['bdry_p2bv_cpi_ln'].apply(np.exp)
quant_rtns

,rtn_1yr_avg,rtn_2yr_avg,max_p2bv_cpi_ln,min_p2bv_cpi_ln,bdry_p2bv_cpi_ln,bdry_p2bv_cpi
q_20,,,,,,
0.0,1.237627,1.340971,1.020769,0.992315,1.020780,2.775358
1.0,1.234013,1.484756,1.029128,1.020791,1.029151,2.798689
2.0,1.255942,1.488818,1.034303,1.029174,1.034333,2.813229
3.0,1.292175,1.501132,1.040573,1.034363,1.040575,2.830844
4.0,1.208937,1.358061,1.046128,1.040577,1.046146,2.846659
5.0,1.031941,1.314347,1.050960,1.046164,1.050963,2.860404
6.0,1.012722,1.245660,1.055121,1.050965,1.055153,2.872416
7.0,1.096507,1.280178,1.057914,1.055186,1.057918,2.880369
8.0,1.109714,1.323340,1.060400,1.057922,1.060402,2.887531


In [235]:
data[['q_20', 'p2bv_cpi_ln']].groupby( ['q_20'], axis=0 ).max().transpose()

q_20,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0
p2bv_cpi_ln,1.020769,1.029084,1.034303,1.04056,1.046241,1.050999,1.055195,1.05794,1.060415,1.062232,1.064666,1.067736,1.071113,1.073738,1.076661,1.080312,1.084473,1.088105,1.094119,1.112697


In [232]:
data['p2bv_cpi_ln'].loc[ data.q_20 == 0 ].max()
data['p2bv_cpi_ln'].loc[ data.q_20 == 1 ].min()

1.020790545215486

### Plot log data

In [226]:
import matplotlib.pyplot as plt
import numpy as np

# For interactive Matplotlib pop-out window
%matplotlib qt

#x = data.loc[~data.bv_a.isnull(), 'eoq_dt']
#y = data.loc[~data.bv_a.isnull(), 'bv_a_cpi_ln']

x1 = data.loc[~data.bv_b_ln_trend.isnull(), 'date']
y1 = data.loc[~data.bv_b_ln_trend.isnull(), 'bv_b_ln_trend']

x2 = data.loc[~data.bv_b_ln_trend.isnull() & ~data.bv_a.isnull(), 'date']
y2 = data.loc[~data.bv_b_ln_trend.isnull() & ~data.bv_b.isnull(), 'bv_b_cpi_ln']

x3 = data.loc[~data.b_cpi_adj_cl_ln.isnull() & (data.date >= '2010-05-08'), 'date']
y3 = data.loc[~data.b_cpi_adj_cl_ln.isnull() & (data.date >= '2010-05-08'), 'b_cpi_adj_cl_ln']

x4 = data.loc[~data.mean_p2bv_ln.isnull(), 'date']
y4 = data.loc[~data.mean_p2bv_ln.isnull(), 'mean_p2bv_ln']

x5 = data.loc[~data.median_p2bv_ln.isnull(), 'date']
y5 = data.loc[~data.median_p2bv_ln.isnull(), 'median_p2bv_ln']

plt.plot(x1, y1, color='black', linewidth=1)
plt.scatter(x2, y2, color='red', s=1)
plt.scatter(x3, y3, color='gold', s=1)
plt.plot(x4, y4, color='gray', linewidth=1)
plt.plot(x5, y5, color='silver', linewidth=1)
plt.show()

In [185]:
#data.loc[ ~data.bv_a.isnull() & ~data.bv_a_ln_trend.isnull()]# .index.values #.tail(20)
data.loc[ data.slope_b.isnull() ].tail()

,date,cpi,eoq_dt,release_dt_x,bv_a,bv_b,bv_a_cpi,bv_b_cpi,bv_a_cpi_ln,bv_b_cpi_ln,release_dt_y,slope_b,intercept_b,bv_b_ln_trend,Adj Close,b_cpi_adj_cl,b_cpi_adj_cl_ln,p2bv_cpi_ln,mean_p2bv_ln,median_p2bv_ln,b_cpi_adj_cl_ffill,rtn_1yr,rtn_1yr_avg,rtn_1yr_med,q_20
3685,2010-05-03,218.025355,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,78.440002,107.212855,4.674816,NaN,NaN,NaN,107.212855,1.016895,1.007564,1.003870,NaN
3686,2010-05-04,218.030806,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,76.610001,104.708965,4.651185,NaN,NaN,NaN,104.708965,1.031695,1.006427,1.003000,NaN
3687,2010-05-05,218.036258,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,76.699997,104.829349,4.652334,NaN,NaN,NaN,104.829349,1.006105,1.005352,1.001765,NaN
3688,2010-05-06,218.041710,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,74.589996,101.942967,4.624414,NaN,NaN,NaN,101.942967,1.041577,1.004109,1.000913,NaN
3689,2010-05-07,218.047161,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,74.410004,101.694427,4.621973,NaN,NaN,NaN,101.694427,1.044122,1.002810,0.999750,NaN


In [169]:
#eoq_dates.info()
#data.info()
#data.loc[ (data.date > '2021-06-20') & (data.date < '2021-07-25') ]
#data.loc[ ~data.p2bv_cpi_ln.isnull() ]

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [47]:
data['rtn_1yr'].sort_values(ascending=False)
data[['date','b_cpi_adj_cl','bv_b_cpi_trend','price_to_trend_bv','cpi','q_20']].loc[ data.date > '2023-01-14' ].head(50)

,date,b_cpi_adj_cl,bv_b_cpi_trend,price_to_trend_bv,cpi,q_20
8708,2023-01-15,NaN,258.029030,NaN,299.170000,NaN
8709,2023-01-16,NaN,258.093516,NaN,299.190034,NaN
8710,2023-01-17,322.081796,258.158019,1.247615,299.210068,5.0
8711,2023-01-18,315.350220,258.222538,1.221234,299.230102,4.0
8712,2023-01-19,312.189137,258.287073,1.208691,299.250136,4.0
8713,2023-01-20,316.913694,258.351624,1.226676,299.270170,4.0
8714,2023-01-21,NaN,258.416192,NaN,299.290204,NaN
8715,2023-01-22,NaN,258.480775,NaN,299.310238,NaN
8716,2023-01-23,317.412470,258.545375,1.227686,299.330271,4.0
8717,2023-01-24,318.290964,258.609991,1.230776,299.350305,4.0


In [76]:
pd.set_option("display.max_rows",130)
pd.set_option("display.max_columns",130)